<div class="alert alert-block alert-warning">
<b>Disclaimer:</b> The main objective of the jupyter notebooks is to show how to use the models of the <i>QENS library</i> by
    
- building a fitting model: composition of models, convolution with a resolution function
- setting and run the fit   
- extracting and displaying information about the results

These steps have a minimizer-dependent syntax. That's one of the reasons why different minimizers have been used in the notebooks provided as examples.
But, the initial guessed parameters might not be optimal, resulting in a poor fit of the reference data.
</div>

<h1> Example: Jump sites log norm diffusion model fitted with scipy </h1>

<a id='Table of Contents'></a><h1>Table of Contents</h1>

- <a href='#introduction'>Introduction</a>  
- <a href='#imports'>Importing libraries</a>
- <a href='#anim_plot'>Plot of the fitting model</a>
- <a href='#ref_data'>Creating reference data</a>
- <a href='#fitting'>Setting and fitting</a>
- <a href='#plot'>Plotting the results</a>  

(<a href='#Table of Contents'>Top</a>)<a id='introduction'></a><h2>Introduction</h2>
<div class="alert alert-block alert-info">
The objective of this notebook is to show how to use one of the models of 
the <a href="https://github.com/QENSlibrary/QENSmodels">QENSlibrary</a>, <b>sqwJumpSitesLogNormDist</b>, to perform some fits. 

<a href="https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html">scipy.optimize.curve_fit</a> is used for fitting.
</div>

### Physical units

For information about unit conversion, please refer to the jupyter notebook called `Convert_units.ipynb` in the `tools` folder.

The dictionary of units defined in the cell below specify the units of the refined parameters adapted to the convention used in the experimental datafile.

In [ ]:
# Units of parameters for selected QENS model and experimental data

dict_physical_units = {'scale': "unit_of_signal.ps", 'center': "1/ps", 'radius': 'Angstrom', 'resTime': 'ps'}

(<a href='#Table of Contents'>Top</a>)<a id='imports'></a><h2>Importing libraries</h2>

In [ ]:
# install QENSmodels (if not already installed)
import pkgutil
import sys

if not pkgutil.find_loader("QENSmodels"):
    buttonY = pnw.Button(name='Yes', button_type='success')
    buttonN = pnw.Button(name='No', button_type='danger')
    choice_installation = panel.Column("Do you want to install the QENSmodels' library?", panel.Row(buttonY, buttonN))
    display(choice_installation)

In [ ]:
if not pkgutil.find_loader("QENSmodels"):
    if buttonY.clicks>0:
        !{sys.executable} -m pip install git+https://github.com/QENSlibrary/QENSmodels#egg=QENSmodels
    elif buttonN.clicks>0:
        print("You will not be able to run some of the remaining parts of this notebook")

In [ ]:
# import python modules for plotting, fitting
from __future__ import print_function
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# for interactive plot
from pandas import DataFrame
import panel.widgets as pnw
import panel as pn
pn.extension()

# import model from QENS library
import QENSmodels

(<a href='#Table of Contents'>Top</a>)<a id='anim_plot'></a><h2>Plot of the fitting model</h2>

The widget below shows the peak shape function imported from QENSmodels where the function's parameters can be varied.

In [ ]:
def mplplot(df, **kwargs):
    fig = df.plot(legend=False).get_figure()
    plt.grid()
    plt.ylabel('jump sites log normal distribution')
    plt.xlabel('x')
    plt.close(fig)
    return fig

def jump_log_norm(q=1, scale=1., center=1., Nsites=3, radius=1., resTime=1., sigma=1., view_fn=mplplot):
    xs = np.linspace(-10,10,100)
    ys = QENSmodels.sqwJumpSitesLogNormDist(xs, q, scale, center, Nsites, radius, resTime, sigma)
    df = DataFrame(dict(y=ys), index=xs)
    return view_fn(df, q=q, scale=scale, center=center, Nsites=Nsites, radius=radius, resTime=resTime, sigma=sigma)

q = pnw.FloatSlider(name='q', value=1, start=.1, end=10)
scale  = pnw.FloatSlider(name='scale', value=5, start=1., end=10)
center = pnw.FloatSlider(name='center', value=5, start=0, end=10)
Nsites = pnw.IntSlider(name='Nsites', value=3, start=2, end=10)
radius = pnw.FloatSlider(name='radius', value=1, start=1, end=10)
resTime = pnw.FloatSlider(name='resTime', value=1, start=1, end=10)
sigma = pnw.FloatSlider(name='sigma', value=1, start=0.1, end=10)

def on_click(event):
    """Reset the interactive plots to inital values."""
    print("reset values")
    q.value = 1
    scale.value = 5
    center.value = 5
    Nsites.value = 3
    radius.value = 1
    resTime.value = 1
    sigma.value = 1 
    
reset_button = pnw.Button(name='Reset')

widgets    = pn.Column("#### sqwJumpSitesLogNormDist", q, scale, center, Nsites, radius, resTime, sigma, reset_button)
jump_log_norm_panel = pn.Row(jump_log_norm(q.value, scale.value, center.value, Nsites.value, radius.value, resTime.value, sigma.value), widgets)

def update(event):
    jump_log_norm_panel[0] = jump_log_norm(q.value, scale.value, center.value, Nsites.value, radius.value, resTime.value, sigma.value)
    
q.param.watch(update, 'value')
scale.param.watch(update, 'value')
center.param.watch(update, 'value')
Nsites.param.watch(update, 'value')
radius.param.watch(update, 'value')
resTime.param.watch(update, 'value')
sigma.param.watch(update, 'value')

reset_button.param.watch(on_click, 'clicks')

jump_log_norm_panel

(<a href='#Table of Contents'>Top</a>)<a id='ref_data'></a><h2>Creating reference data</h2>

**Input:** the reference data for this simple example correspond to sqwJumpSitesLogNormDist with added noise.

The fit is performed using `scipy.optimize.curve_fit`. <br> The example is based on implementations from https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html

In [ ]:
# Creation of reference data
xx = np.linspace(-10,10,100)
sqw_jump_sites_noisy = QENSmodels.sqwJumpSitesLogNormDist(xx, 0.89, 1, 0.3, 5, 2, 0.45, 0.25)*(1. + 0.04*np.random.normal(0,1,100)) + 0.02*np.random.normal(0,1,100)

plt.plot(xx, sqw_jump_sites_noisy, label='reference data')
plt.xlabel('x')
plt.grid()
plt.legend()
plt.show()

(<a href='#Table of Contents'>Top</a>)<a id='fitting'></a><h2>Setting and fitting</h2>

In [ ]:
# From https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html
# perform fit with initial guesses scale=0.95, center=0.2, radius=2, resTime=0.45, sigma=0.25
# Nsites=5 and q =0.89 are fixed

def func_to_fit(xx, scale, center, radius, resTime, sigma):
    return QENSmodels.sqwJumpSitesLogNormDist(xx, 0.89, scale, center, 5, radius, resTime, sigma)
plt.plot(xx, sqw_jump_sites_noisy, 'b-', label='reference data')
plt.plot(xx, QENSmodels.sqwJumpSitesLogNormDist(xx, 0.89, scale=0.95, center=0.2, Nsites=5, radius=2, resTime=0.45, sigma=0.25), 'r-', label='model with initial guesses')
plt.xlabel('x')
plt.grid()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

popt, pcov = curve_fit(func_to_fit, xx, sqw_jump_sites_noisy, p0=[0.95, 0.2, 2, 0.45, 0.25], 
                       bounds=((0.1, -2, 0.1, 0.1, 0.1), (5, 2, 5, 11, 1)))

(<a href='#Table of Contents'>Top</a>)<a id='plot'></a><h2>Plotting the results</h2>

In [ ]:
# Calculation of the errors on the refined parameters:
perr = np.sqrt(np.diag(pcov))

print('Values of refined parameters:')
print('scale:'  , popt[0],'+/-', perr[0], dict_physical_units['scale'])
print('center :', popt[1],'+/-', perr[1], dict_physical_units['center'])
print('radius'  , popt[2],'+/-', perr[2], dict_physical_units['radius'])
print('resTime' , popt[3],'+/-', perr[3], dict_physical_units['resTime'])
print('sigma'   , popt[4],'+/-', perr[4])

# Comparison of reference data with fitting result
plt.plot(xx, sqw_jump_sites_noisy, 'b-', label='reference data')
plt.plot(xx, func_to_fit(xx, *popt), 'g--', label='fit: %5.3f, %5.3f, %5.3f, %5.3f, %5.3f' % tuple(popt))
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left'), borderaxespad=0.)
plt.xlabel('x')
plt.grid()
plt.show()